# Task 2: Break XOR (Brute Force)

## Brute-Force Script

import sys

def xor_decrypt(data, key):
    """
    # Performs single-byte XOR decryption.
    # Why XOR works for both encryption and decryption:
    # Because (P ⊕ K) ⊕ K = P (XOR is self-inverse).
    """
    try:
        return bytes([byte ^ key for byte in data])
    except:
        """
        # Possible exceptions:
        # - TypeError if key is not an integer
        # - Issues if data is not in bytes format
        """
        return None


def score_plaintext(candidate):
    """
    # Simple scoring function to determine if output resembles English text.
    # Why scoring is needed:
    # Because brute force generates 256 outputs — we must rank them.
    """

    if candidate is None:
        return -1

    printable = sum(1 for b in candidate if 32 <= b <= 126 or b in (9, 10, 13))
    letters = sum(1 for b in candidate if (65 <= b <= 90) or (97 <= b <= 122))
    spaces = candidate.count(32)

    score = printable + letters + (spaces * 2)
    return score


def brute_force_xor(ciphertext):
    """
    # Tries all 256 possible single-byte keys (0x00 - 0xFF).
    # Why 256 keys?
    # Because a single byte has 2^8 = 256 possible values.
    """

    results = []

    for key in range(256):
        decrypted = xor_decrypt(ciphertext, key)
        score = score_plaintext(decrypted)
        results.append((score, key, decrypted))

    # Sort from highest score to lowest
    results.sort(reverse=True, key=lambda x: x[0])

    return results


def main():
    """
    # Checks for correct command-line arguments.
    # Why use len(sys.argv)?
    # To ensure user provides required ciphertext filename.
    """

    if len(sys.argv) != 2:
        print("Usage: python3 bruteforce_xor.py <ciphertext_file>")
        sys.exit(1)

    filename = sys.argv[1]

    try:
        with open(filename, "rb") as f:
            ciphertext = f.read()
    except:
        print("Error: Could not open file.")
        sys.exit(1)

    print(f"Brute-forcing single-byte XOR on {filename}...")
    print("Trying all 256 possible keys...\n")

    results = brute_force_xor(ciphertext)

    print("Top 5 candidate keys:\n")

    for i in range(5):
        score, key, plaintext = results[i]
        print(f"{i+1}) Key: 0x{key:02x} | Score: {score}")
        print("Preview:", plaintext[:100].decode(errors="replace"))
        print()

    best_score, best_key, best_plaintext = results[0]

    with open("recovered_text.txt", "wb") as out:
        out.write(best_plaintext)

    print(f"[+] Recovered Key: 0x{best_key:02x}")
    print("[+] Decrypted plaintext saved to recovered_text.txt")


if __name__ == "__main__":
    main()
